## Explainer notebook: Computing Identifiable Distances at a Large Scale

A walkthrough of the computations, algorithms, and results for this project.

#### Data

The Tasic et al. (Nature 2018) single-cell RNA-seq dataset is used, containing gene expression profiles from 23,822 cells isolated from the adult mouse visual cortex. The dataset is used in a preprocessed form, with PCA reducing the dimensionality to 50 components. The PCA-reduced data is available via the accompanying GitHub repository [(Berens and Kobak (2019))](https://github.com/berenslab/rna-seq-tsne/tree/master/data/tasic-preprocessed). 

The dataset contains 133 unique cell classes.

#### Motivation

Deep latent variable models (DLVMs) are powerful tools for uncovering low-dimensional structure in high-dimensional data. The goal is not merely to compress data, but to learn representations that reflect the underlying mechanisms of the observed phenomena.

However, in VAEs and similar deep generative models, the latent variables themselves are not identifiable. Multiple different latent representations can produce the same distribution of observed data. As a result, the learned latent variables are not unique and can vary significantly across training runs, making interpretation difficult [Hauberg (2025)](https://www2.compute.dtu.dk/~sohau/weekendwithbernie/Differential_geometry_for_generative_modeling.pdf).

Recent work has shown that while the latent variables themselves are not identifiable, certain geometric relationships between them, such as distances, angles, and volumes, can be. When defined using the pullback metric via the decoder these relationships can be statistically identifiable under mild assumptions [Syrota et al. (2025)](https://github.com/mustass/identifiable-latent-metric-space).

#### Objective and Aim

Under the manifold hypothesis, it is assumed that the data lie near a lower-dimensional manifold embedded in a highdimensional space. A variational autoencoder (VAE) or an ensemble VAE is used to learn this manifold.

The objective is to produce distance matrices and implement methods to compute geodesic distances.

* The first experiment is set up with a single VAE. One encoder, one decoder.
* The second experiment is set up using an ensemble of VAEs with 10 decoders.

#### Single VAE

Two variational autoencoders are trained independently using different random seeds, as implemented in `src/single_decoder/vae_train.py`. Each model maps the 50-dimensional PCA-reduced RNA-seq data to a 2-dimensional latent space. Training is performed for 100 epochs using the Adam optimizer with a learning rate of 0.001 and a batch size of 64.

The objective is to compute geodesic distances between representative latent points and compare the resulting distance matrices across models. This serves to test the hypothesis that geodesic distances—defined through the decoder—are more stable across random initializations than Euclidean distances. To enable this comparison, 133 representative points (one per class) are selected, yielding 8778 pairwise distances (`src/select_representative_pairs.py`).


**Spline Initialization**

Geodesics are parameterized as cubic splines, following the formulation in Syrota et al. (2025). These splines are subject to $C^2$ continuity constraints at internal knots, ensuring smoothness by enforcing the continuity of position, velocity, and acceleration. For each pair of adjacent spline segments, this is enforced through three linear constraints defined by vectors $c^0, c^1, c^2$. corresponding respectively to continuity of the function, its first derivative, and second derivative at the joint. These constraints are expressed compactly as a linear system:

$$A \xi^\top = \mathbf{0}$$

where $A$ stacks the constraint vectors across all segments, and $\xi$ denotes the full set of spline parameters.  The null space of 
$A$ defines the set of all spline coefficient vectors that satisfy the continuity constraints. Optimization is carried out over this subspace to minimize the geodesic energy. This basis is implemented in the `GeodesicSplineBatch` class.

Syrota et al. (2025) initialize splines using zero-valued coefficients corresponding to a straight-line path. In this project, this is further extended by using a Dijkstra-initialized path as the starting point, inspired by the graph-based initialization strategy proposed by [Detlefsen et al. (2022)](https://www.nature.com/articles/s41467-022-29443-w), adapted here for single VAE geometry. A Dijkstra graph search  (via SciPy) is run over a $200 \times 200$ grid in latent space using $k=8$ nearest-neighbor connectivity, with edge weights given by Euclidean distance. The resulting shortest paths are fitted with 4-segment per spline cubic splines using the LBFGS optimizer to minimize mean squared error between the spline and the Dijkstra path on the grid. The spline is evaluated at uniformly spaced parameter values $t \in [0,1]$, and the fitted coefficients are used as initialization for subsequent energy-based optimization that follows in the next paragraph. This initialization procedure is implemented in `src/single_decoder/init_spline.py`.

<div style="display: flex; justify-content: space-around; align-items: center;">
  <div style="text-align: center;">
    <img src="../src/plots/splines_init_dijkstra_seed12.png" alt="Splines Init Dijkstra Seed 12" style="max-width: 45%; height: auto;">
    <p style="text-align: center;"><em>Dijkstra-initialized cubic splines in the latent space of a single VAE (seed 12). Each dashed line represents a shortest path between two latent points, computed over a 200x200 grid using k=8 nearest-neighbor connectivity and Euclidean edge weights. Solid curves show the smooth fitted cubic splines (four segments) used for initialization. The background shows the full latent embedding. Since the initialization is purely graph-based in latent space, splines are not prohibited from entering regions of low data density. This is later discouraged in the optimization. </em></p>
</div>


**Energy Optimization**

Energy optimization is carried out over the same spline parameterization described above, using the $\omega$ coefficients defined in the null space basis. The script `src/single_decoder/optimize_energy_batched.py`performs this optimization in batches of splines.

The optimization is performed using the Adam optimizer with 500 steps and a learning rate of 0.001. Each initialized spline is optimized using the same number of cubic segments as in the initialization. The energy is computed over 2000 uniformly spaced time steps (same as each spline is initialized with) using decoded means, providing an approximation of the path length in data space. This avoids explicit Jacobian computations.

The optimized splines are saved, along with the approximate geodesic lengths. Below are examples of initial (dashed) and optimized (solid) splines, visualized on density-based latent backgrounds.

<div style="display: flex; justify-content: space-around; align-items: center;">
  <div style="text-align: center;">
    <img src="../src/plots/density_illustration_examples12.png" alt="Initial and optimized splines seed 12" style="max-width: 80%; height: auto;">
    <p style="text-align: center;"><em>Seed 12</em></p>
  </div>
  <div style="text-align: center;">
    <img src="../src/plots/density_illustration_examples123.png" alt="Initial and optimized splines seed 123" style="max-width: 80%; height: auto;">
    <p style="text-align: center;"><em>Seed 123</em></p>
  </div>
</div>
<p style="text-align: center; margin-top: 1em;"><em>The initial paths are optimized and appear to be pushed toward regions of higher density in the latent space. For example the orange curve on the right in seed 123 that seems to be optimized to go through multiple smaller clusters on the way between the endpoints or the blue curves in seed 123 that visits a closer, nearby cluster first, rather than following a purely linear path between the endpoints. While the optimized paths generally follow denser regions, some paths still appear to partially cross through lower density areas.</em><p>



**Geodesic Distance Matrices VAE**

Approximate geodesic lengths from optimized splines are assembled into pairwise distance matrices between chosen cluster representative points. The matrices below, shown for two different seeds, exhibit similar global structure despite being based on separate model initializations.

Note that only one point per cluster is used as representative, so the resulting distances should not be interpreted as capturing full cluster-to-cluster structure — they reflect specific point-to-point geodesics.

<div style="display: flex; justify-content: space-between;">
  <div style="flex: 0 0 49%; text-align: center;">
    <img src="../src/plots/geodesic_distance_seed12_p133.png" alt="Seed 12" style="width: 80%; height: auto;">
    <p style="text-align: center;"><em>Seed 12</em></p>
  </div>
  <div style="flex: 0 0 49%; text-align: center;">
    <img src="../src/plots/geodesic_distance_seed123_p133.png" alt="Seed 123" style="width: 80%; height: auto;">
    <p style="text-align: center;"><em>Seed 123</em></p>
  </div>
</div>
<p style="text-align: center; margin-top: 1em;"><em>Although the matrices look to have similar structure, the geodesic distances for the comparable points differ numerically.</em></p>


To reduce sensitivity to initialization observed in single-decoder geodesics, the next step is to explore ensembles of VAEs.

#### Ensemble VAE

While individual latent encodings in VAEs are non-identifiable, the decoder induces a Riemannian geometry on latent space via the pullback of the Euclidean metric in data space. However, this geometry depends on the specific decoder instance and may vary with model initialization.

To reduce this sensitivity, geodesics are computed across an ensemble of independently trained decoders. This ensemble approach aims to reduce variance in the induced metric structure, yielding more consistent approximations of geodesic paths.

The models are trained for 500 epochs using the Adam optimizer with a learning rate of 0.001 and batch size of 64. The `EVAE` class can be seen in the `src/train.py` file together with the training loop.

**Latent spaces**

To examine the effect of ensembling on the learned geometry, the latent representations and decoder-induced uncertainty are visualized across different seeds.

<div style="display: flex; justify-content: space-between;">
  <div style="flex: 0 0 49%; text-align: center;">
    <img src="../experiment/plots/latent_plot_seed12.png" alt="Seed 12" style="width: 80%; height: auto;">
    <p style="text-align: center;"><em>Seed 12 EVAE 10 decoders</em></p>
  </div>
  <div style="flex: 0 0 49%; text-align: center;">
    <img src="../experiment/plots/latent_plot_seed123.png" alt="Seed 123" style="width: 80%; height: auto;">
    <p style="text-align: center;"><em>Seed 123 EVAE 10 decoders</em></p>
  </div>
</div>
<p style="text-align: center; margin-top: 1em;"><em>It is noted that the latent spaces look very similar across seeds. This unfortunately hints that the euclidean distances will be much alike, and thereby that the geodesic approximated distances perhaps will not be a big improvement over euclidean for this dataset. The similar latent spaces might most likely be because of the pca reduced data.</em></p>



**Initializing Splines - Ensemble methods**

To initialize geodesic splines for ensemble VAEs, two graph-based methods are used:

* Euclidean Graph: Edge weights correspond to latent-space distances.
* Entropy-Weighted Graph: Edge weights reflect decoder disagreement (l2-norm of the standard deviation across ensemble means), inspired by Detlefsen et al. (2022).

For each selected representative pair, shortest paths are computed using Dijkstra’s algorithm over a $200\times200$ latent grid with $k=8$ connectivity. These paths are then fitted with 4-segment cubic splines using LBFGS, following the continuity-constrained formulation as before (Syrota et al. (2025)). The procedure is implemented in `src/init_spline_ensemble.py`, with the initialization method toggled via the `--use-entropy` flag.

Both approaches have approximately the same computational cost.

<div style="display: flex; justify-content: space-between;">
  <div style="flex: 0 0 49%; text-align: center;">
    <img src="../experiment/splines_init_model_seed12/spline_plot_init_euclidean_10.png" alt="Seed 12 euclidean init" style="width: 80%; height: auto;">
    <p style="text-align: center;"><em>Initialized using euclidean based graph</em></p>
  </div>
  <div style="flex: 0 0 49%; text-align: center;">
    <img src="../experiment/splines_init_model_seed12/spline_plot_init_entropy_10.png" alt="Seed 12 entropy init" style="width: 80%; height: auto;">
    <p style="text-align: center;"><em>Initialized using entropy weighted graph</em></p>
  </div>
</div>
<p style="text-align: center; margin-top: 1em;"><em>Both are for the EVAE model with seed 12. The entropy-based initialization produces splines that visually follow the latent structure more closely than the Euclidean-based initialization. For the entropy initialization it might be redundant to do further optimization.</em></p>


**Optimization - In Ensembles**

Spline optimization in the ensemble setting follows the same procedure as for the single VAE, implemented in `src/optimize.py`. The key difference lies in the evaluation of the geodesic energy, which is now estimated using Monte Carlo sampling over decoder pairs.

At each step, the energy is computed as the average squared distance between consecutive reconstructions along the spline, using independently sampled decoder pairs from the ensemble. For $M$ samples, this gives the following estimator:

$$
\mathcal{E}(c) \approx \sum_{i=0}^{N} \mathbb{E}_{l,k} \left\| f_l(c(t_i)) - f_k(c(t_{i+1})) \right\|^2
$$

Here, $f_{l}, f_{k}$ are decoders sampled from the ensemble, and $c(t)$ denotes the current spline.

Empirically, increasing $M$ beyond 2 (e.g., to 10) does not significantly change the resulting paths. Therefore, $M=2$ is chosen to reduce computational cost.

Some artifacts such as spline self-intersections or sharp turns occasionally occur, especially when using large batch sizes or extended optimization (e.g., >500 steps). While $C^2$ continuity is still enforced via the nullspace formulation, no additional regularization or curvature constraints were implemented to address these artifacts. This remains an area for future work to reveal if this can be done. Increasing the number of spline segments was also found to have limited effect on this issue.


<div style="display: flex; justify-content: space-between;">
  <div style="flex: 0 0 49%; text-align: center;">
    <img src="../experiment/splines_opt_model_seed123/spline_plot_both_euclidean_133.png" alt="Seed 123 euclidean init and optimized" style="width: 80%; height: auto;">
    <p style="text-align: center;"><em>Seed 123 Euclidean initialized distances</em></p>
  </div>
  <div style="flex: 0 0 49%; text-align: center;">
    <img src="../experiment/splines_opt_model_seed123/spline_plot_both_entropy_133.png" alt="Seed 123 entropy init and optimized" style="width: 80%; height: auto;">
    <p style="text-align: center;"><em>Seed 123 Entropy initialized distances</em></p>
  </div>
</div>
<p style="text-align: center; margin-top: 1em;"><em>Optimized geodesics for seed 123, initialized using euclidean (L) and entropy (R). It is noted how the optimization turns out different depending on the initialization.</em></p>


**Geodesic Distances vs Euclidean Distances**

The geodesic distance matrices presented here below show approximate geodesic lengths computed between pairs of cluster representative points, based on optimized splines initialized using either Euclidean or entropy-weighted graphs. The results are shown for two different model seeds, and notably, the matrices exhibit similar global structure across these seeds despite separate initializations. It is important to note that only one representative point per cluster is used, so these distances represent specific point-to-point geodesics rather than full cluster-to-cluster relationships, limiting broader interpretability. Some white points are visible in the matrices, indicating pairs for which distances were not computed, likely due to low grid resolution.

<div style="display: flex; justify-content: space-between;">
  <div style="flex: 0 0 49%; text-align: center;">
    <img src="../experiment/plots/geodesic_matrix_seed12_entropy_133.png" alt="Seed 12 entropy init" style="width: 80%; height: auto;">
    <p style="text-align: center;"><em>Seed 12 Entropy distances</em></p>
  </div>
  <div style="flex: 0 0 49%; text-align: center;">
    <img src="../experiment/plots/geodesic_matrix_seed123_entropy_133.png" alt="Seed 123 entropy init" style="width: 80%; height: auto;">
    <p style="text-align: center;"><em>Seed 123 Entropy distances</em></p>
  </div>
</div>

<div style="display: flex; justify-content: space-between;">
  <div style="flex: 0 0 49%; text-align: center;">
    <img src="../experiment/plots/geodesic_matrix_seed12_euclidean_133.png" alt="Seed 12 euc init" style="width: 80%; height: auto;">
    <p style="text-align: center;"><em>Seed 12 Euclidean distances</em></p>
  </div>
  <div style="flex: 0 0 49%; text-align: center;">
    <img src="../experiment/plots/geodesic_matrix_seed123_euclidean_133.png" alt="Seed 123 euc init" style="width: 80%; height: auto;">
    <p style="text-align: center;"><em>Seed 123 Euclidean distances</em></p>
  </div>
</div>

Next, we visualize the pairwise Euclidean distance matrices computed directly in the latent space for two different model seeds.

It is important to note that a direct comparison between Euclidean and geodesic distance matrices is not meaningful, as they are measured in fundamentally different spaces and with different geometric assumptions.

Nevertheless, the Euclidean distance matrices shown here are remarkably similar across the two seeds. This consistency likely stems from the latent spaces themselves being similar across initializations, as observed in earlier analyses.

While this similarity supports the stability of the latent representation, it also implies that the advantage of computing geodesic distances over simpler Euclidean distances is not clearly demonstrated for this particular dataset. The latent space geometry here appears close to Euclidean, which reduces the necessity for more complex geodesic computations.

<div style="display: flex; justify-content: space-between;">
  <div style="flex: 0 0 49%; text-align: center;">
    <img src="../experiment/plots/euclidean_dist_matrix_seed12__133.png" alt="Seed 12" style="width: 80%; height: auto;">
    <p style="text-align: center;"><em>Seed 12 Euclidean distances</em></p>
  </div>
  <div style="flex: 0 0 49%; text-align: center;">
    <img src="../experiment/plots/euclidean_dist_matrix_seed123__133.png" alt="Seed 123" style="width: 80%; height: auto;">
    <p style="text-align: center;"><em>Seed 123 Euclidean distances</em></p>
  </div>
</div>

**CoV analysis**

Stability of geodesic distances was evaluated across different numbers of decoders using CoV on six models with ten decoders each. The average CoV of geodesic distances decreases significantly from one to two decoders and then stabilizes, indicating increased robustness with more decoders. Euclidean distances show consistently higher and unchanged CoV. These results only suggest ensemble geodesic distances provide more stable estimates across multiple decoders, but the limited sample size means conclusions are not statistically definitive.

<div style="display: flex; justify-content: space-around; align-items: center;">
  <div style="text-align: center;">
    <img src="../experiment/plots/cov_plot_15_alldec.png" alt="Splines Init Dijkstra Seed 12" style="max-width: 45%; height: auto;">
    <p style="text-align: center;"><em>CV analysis using 6 models trained with 10 decoders. Evaluated using 105 different splines. Evaluating geodesic and euclidean distances with increasing number of decoders. The results are not statistically valid to conclude on. (The geodesics' splines are just initialized using linear interpolation between points to not introduce different initialization methods in CV analysis). </em></p>
</div>

**Report Conclusion**

The distance matrices for geodesic for different model initializations (across different spline initialization methods) look as similar as the Euclidean distance matrices when compared across seeds, implying that the advantage of computing geodesics — despite their higher computational cost — is not clearly demonstrated for this dataset that geodesics necessarily provide better or more informative geometric representations than simple Euclidean distances. Given that the Coefficient of Variation (CoV) analysis is not statistically conclusive, it can only be suggested that geodesics may offer more stable estimates due to ensemble averaging. 